In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from qutip import *
from scipy.optimize import curve_fit

plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rc('figure', titlesize=20)

In [19]:
def readFile(filename):
    timeTickFirst, timeTickLast, x, ion3, ion3_raw, ion3_bottom, ion3_top, ion4, ion4_raw, ion4_bottom, \
    ion4_top, ion5, ion5_raw, ion5_bottom, ion5_top, ion2, ion2_raw, ion2_bottom, ion2_top, ion6, ion6_raw, \
    ion6_bottom, ion6_top\
      = np.loadtxt(filename, comments='#', delimiter='\t', converters=None,
                   skiprows=0,unpack=True, encoding='latin1')
    return x, ion4

In [20]:
df_dep = pd.read_excel('data_VAET_dep.xlsx')
df_dep

filename_kappa0p2 filename_kappa0p1 filename_kappa0p0
0       kappa0p2_001      kappa0p1_001         kappa_0p0
1       kappa0p2_002      kappa0p1_002         kappa_0p1
2       kappa0p2_003      kappa0p1_003         kappa_0p2
3       kappa0p2_004      kappa0p1_004         kappa_0p3
4       kappa0p2_005      kappa0p1_005               NaN
5       kappa0p2_006      kappa0p1_006               NaN
6       kappa0p2_007      kappa0p1_007               NaN
7       kappa0p2_008      kappa0p1_008               NaN
8       kappa0p2_009      kappa0p1_009               NaN
9       kappa0p2_010      kappa0p1_010               NaN
10      kappa0p2_011      kappa0p1_011               NaN
11      kappa0p2_012      kappa0p1_012               NaN
12      kappa0p2_013      kappa0p1_013               NaN
13      kappa0p2_014      kappa0p1_014               NaN
14      kappa0p2_015      kappa0p1_015               NaN
15      kappa0p2_016      kappa0p1_016               NaN
16      kappa0p2_017      kappa0p1_017               NaN
17      kappa0p2_018      kappa0p1_018               NaN
18      kappa0p2_019      kappa0p1_019               NaN
19      kappa0p2_020      kappa0p1_020               NaN

In [21]:
def get_pops(params, option='c'):
    Nmax = params['Nmax']
    H = params['eps']/2 * tensor(sigmaz(), qeye(Nmax))
    H += params['Delta']/2 * tensor(sigmax(), qeye(Nmax))
    H += params['kappa']/2 * tensor(sigmaz(), (destroy(Nmax) + create(Nmax)))
    H += params['nu'] * tensor(qeye(2), create(Nmax) * destroy(Nmax))
    init_state = tensor(ket2dm(basis(2,0)), thermal_dm(Nmax, params['nbar']))
    tlist = params['tlist']
    e_op = tensor(ket2dm(basis(2,0)), qeye(Nmax))
    e_op2 = tensor(qeye(2), create(Nmax) * destroy(Nmax))
    if option == 'c':
        c_ops = [np.sqrt(params['Gamma'] * (params['nbar'] + 1)) * tensor(qeye(2), destroy(Nmax))]
        if params['nbar'] > 0:
            c_ops += [np.sqrt(params['Gamma'] * params['nbar']) * tensor(qeye(2), create(Nmax))]
    elif option == 'd':
        c_ops = [np.sqrt(params['Gamma']) * tensor(qeye(2), create(Nmax) * destroy(Nmax))]
    result = mesolve(H, init_state, tlist, e_ops=[e_op, e_op2], c_ops=c_ops)
    pops = result.expect[0]
    phonons = result.expect[1]
    return pops, phonons


def analyze(data_list):
    expdata_all = np.array(data_list).reshape((len(data_list), -1))
    expdata = np.mean(expdata_all, axis=0)
    
    variance = np.std(expdata_all, axis=0)**2 / len(data_list)
    shotnoise = np.maximum(np.sqrt(expdata_all * (1-expdata_all) / 100), 1e-3)
    variance = variance + np.mean(shotnoise**2, axis=0)
    errorbar = np.sqrt(variance)
    
    return expdata, errorbar

In [22]:
sim_time = 14 * 2*np.pi
steps = 140
tlist = np.linspace(0, sim_time, steps+1)

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.0, 'Gamma': 0.1, 'nbar': 0.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_cool1, _ = get_pops(params, option='c')
pops_sim_dep1, _ = get_pops(params, option='d')

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.1, 'Gamma': 0.1, 'nbar': 0.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_cool2, _ = get_pops(params, option='c')
pops_sim_dep2, _ = get_pops(params, option='d')

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.2, 'Gamma': 0.1, 'nbar': 0.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_cool3, _ = get_pops(params, option='c')
pops_sim_dep3, _ = get_pops(params, option='d')

In [23]:
## Unit Conversion ##
Delta_cm = 100 #cm^-1
Delta_eV = Delta_cm/8065.73 # eV
hbar = 6.62607015e-34/2/np.pi
q = 1.60217663e-19
Delta = Delta_eV*q/hbar # Hz * 2pi
print('Delta: 2 pi '+str(Delta/1e12/2/np.pi)+' THz')
scale = 1e12/(Delta/2/np.pi) #1e12 ps / (freq in Hz)
print(scale)
tlist = tlist*scale

Delta: 2 pi 2.9978554155025736 THz
0.3335717909638933


In [24]:
tlist[-1]/2/np.pi

4.670005073494506

In [25]:
dep_list_1 = []
for fname in df_dep['filename_kappa0p0'][:4]:
    filename = fname
    _, pop = readFile(filename)
    dep_list_1 += [pop]
dep_pop_1, error_dep_1  = analyze(dep_list_1)

dep_list_2 = []
for fname in df_dep['filename_kappa0p1'][:20]:
    filename = fname
    _, pop = readFile(filename)
    dep_list_2 += [pop]
dep_pop_2, error_dep_2  = analyze(dep_list_2)

dep_list_3 = []
for fname in df_dep['filename_kappa0p2'][:20]:
    filename = fname
    _, pop = readFile(filename)
    dep_list_3 += [pop]
dep_pop_3, error_dep_3  = analyze(dep_list_3)

color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']
plt.figure(figsize = (8,2.4))



t_exp = tlist/2/np.pi

plt.errorbar(t_exp[::4], dep_pop_1, yerr=error_dep_1,\
                fmt='o', alpha=0.5,markersize=5, color=color_list[0], ecolor=color_list[0],\
                mfc=color_list[0])
plt.plot(tlist/2/np.pi, pops_sim_dep1, ls = '-', color=color_list[0],label=r'$\kappa = 0$  cm$^{-1}$')
plt.plot(tlist/2/np.pi, pops_sim_cool1, ls = '-.', color=color_list[0])

plt.errorbar(t_exp[::4], dep_pop_2, yerr=error_dep_2,\
                fmt='o', alpha=0.5,markersize=5, color=color_list[1], ecolor=color_list[1],\
                mfc=color_list[1])
plt.plot(tlist/2/np.pi, pops_sim_dep2, ls = '-', color=color_list[1],label=r'$\kappa = 10$ cm$^{-1}$')
plt.plot(tlist/2/np.pi, pops_sim_cool2, ls = '-.', color=color_list[1])

plt.errorbar(t_exp[::4], dep_pop_3, yerr=error_dep_3,\
                fmt='o', alpha=0.5,markersize=5, color=color_list[2], ecolor=color_list[2],\
                mfc=color_list[2])
plt.plot(tlist/2/np.pi, pops_sim_dep3, ls = '-', color=color_list[2],label=r'$\kappa = 20$ cm$^{-1}$')
plt.plot(tlist/2/np.pi, pops_sim_cool3, ls = '-.', color=color_list[2])
plt.legend()
plt.grid(True)
plt.xlabel(r'Time [ps]')
plt.ylabel('Donor pop.')
plt.ylim([-0.05, 1.05])

plt.savefig('Figure5_3.png', dpi=300, bbox_inches='tight')

OSError: kappa_0p0 not found.